# Bronze Layer

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/bookings/data`;--Run-1: 1000 rows. Run-2: 1300 Rows (Incremental Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/customers/data`;--Run-1: 200 rows. Run-2: 220 Rows (Incremental Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/airports/data`;--Run-1: 50 rows. Run-2: 55 Rows (Incremental Load)

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronzevolume/flights/data`;--Run-1: 100 rows. Run-2: 110 Rows (Incremental Load)